In [13]:
%pip install yfinance statsmodels --quiet


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: C:\Users\lenovo ideapad\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [14]:
import numpy as np
import pandas as pd
import yfinance as yf
import statsmodels.api as sm
from statsmodels.tsa.stattools import adfuller
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
plt.style.use("seaborn-v0_8")
from sklearn.preprocessing import StandardScaler
import os


In [15]:
nifty100 = [
"ADANIENT.NS","ADANIPORTS.NS","ASIANPAINT.NS","AXISBANK.NS","BAJAJ-AUTO.NS",
"BAJFINANCE.NS","BAJAJFINSV.NS","BPCL.NS","BHARTIARTL.NS","BOSCHLTD.NS",
"BRITANNIA.NS","CIPLA.NS","COALINDIA.NS","DIVISLAB.NS","DRREDDY.NS",
"EICHERMOT.NS","GRASIM.NS","HCLTECH.NS","HDFC.NS","HDFCBANK.NS",
"HDFCLIFE.NS","HEROMOTOCO.NS","HINDALCO.NS","HINDUNILVR.NS","ICICIBANK.NS",
"ITC.NS","INDUSINDBK.NS","INFY.NS","JSWSTEEL.NS","KOTAKBANK.NS",
"LT.NS","M&M.NS","MARUTI.NS","NESTLEIND.NS","NTPC.NS",
"OIL.NS","ONGC.NS","POWERGRID.NS","RELIANCE.NS","SHREECEM.NS",
"SBIN.NS","SUNPHARMA.NS","TCS.NS","TATACONSUM.NS","TATAMOTORS.NS",
"TATASTEEL.NS","TECHM.NS","TITAN.NS","ULTRACEMCO.NS","UPL.NS",
"WIPRO.NS","COFORGE.NS","BANDHANBNK.NS","MOTHERSUMI.NS","SBILIFE.NS",
"PEL.NS","NTPC.NS","HINDPETRO.NS","VINATIORGA.NS","ADANIGREEN.NS",
"AUROPHARMA.NS","BHEL.NS","CADILAHC.NS","CANBK.NS","CHOLAFIN.NS",
"DLF.NS","GAIL.NS","HAVELLS.NS","IBULHSGFIN.NS","IDEA.NS",
"INDIGO.NS","LTIM.NS","MFSL.NS","MPI.NS","NAUKRI.NS",
"NMDC.NS","PAGEIND.NS","PEL.NS","PNB.NS","RECLTD.NS",
"SUNTV.NS","TATAPOWER.NS","TATACHEM.NS","TVSMOTOR.NS","WELCORP.NS",
"ZOMATO.NS","BALKRISIND.NS","BERGEPAINT.NS","BIOCON.NS","CROMPTON.NS",
"EXIDEIND.NS","GLENMARK.NS","HDFCLIFE.NS","IDFCFIRSTB.NS","INDHOTEL.NS",
"JUBLFOOD.NS","LTI.NS","MCDOWELL-N.NS","NIACL.NS","PNBHOUSING.NS"
]
# Remove duplicates & ensure unique
nifty100 = list(dict.fromkeys(nifty100))
print("Tickers loaded:", len(nifty100))


Tickers loaded: 97


In [16]:
# ===============================
# 1️⃣ Download prices 2019–2025
# ===============================
start_date = "2019-01-01"
end_date = "2025-12-31"   # extended to allow proper OOS + rolling warmup

print("Downloading price data ...")
raw = yf.download(nifty100, start=start_date, end=end_date,
                  group_by="ticker", auto_adjust=True, threads=True)

# Extract Close prices robustly
if ("Close" in raw.columns) or (
    isinstance(raw.columns, pd.MultiIndex) and "Close" in raw.columns.levels[1]
):
    prices = pd.DataFrame()
    for t in nifty100:
        try:
            prices[t] = raw[t]["Close"]
        except Exception:
            pass
else:
    prices = raw.copy()

# Clean data
prices.dropna(axis=1, how='all', inplace=True)
prices.dropna(axis=0, how='any', inplace=True)
prices = prices.loc[:, prices.std() > 0]

print("Final stocks with complete data:", prices.shape[1])
prices.tail()

# ===============================
# 2️⃣ Define selection vs backtest windows
# ===============================
selection_prices = prices.loc["2019-01-01":"2024-12-31"]   # for pair selection
backtest_prices  = prices.loc["2025-01-01":"2025-12-31"]   # OOS backtest


[*********************100%***********************]  96 of 97 completed

10 Failed downloads:
['HDFC.NS', 'MCDOWELL-N.NS', 'ZOMATO.NS', 'PEL.NS', 'MPI.NS', 'IBULHSGFIN.NS', 'LTI.NS', 'CADILAHC.NS', 'MOTHERSUMI.NS']: YFTzMissingError('possibly delisted; no timezone found')
['TATAMOTORS.NS']: YFPricesMissingError('possibly delisted; no price data found  (1d 2019-01-01 -> 2025-12-31) (Yahoo error = "No data found, symbol may be delisted")')


Final stocks with complete data: 87


In [ ]:
from sklearn.cluster import AgglomerativeClustering
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm
import numpy as np
import pandas as pd

# Use only selection window
returns = selection_prices.pct_change().dropna()

# Correlation distance
corr_matrix = returns.corr()
dist_matrix = np.sqrt(2 * (1 - corr_matrix))
tickers = corr_matrix.columns.tolist()

# Agglomerative clustering
n_clusters = 20
clustering = AgglomerativeClustering(
    n_clusters=n_clusters,
    metric="precomputed",
    linkage="average"
)
labels = clustering.fit_predict(dist_matrix.values)

# Generate candidate pairs
selected = []

def is_cointegrated(y, x, pval_threshold=0.05):
    beta = sm.OLS(y, sm.add_constant(x)).fit().params[1]
    spread = y - beta * x
    pval = adfuller(spread)[1]
    return pval < pval_threshold

for cluster_id in np.unique(labels):
    idx = np.where(labels == cluster_id)[0]
    if len(idx) < 2:
        continue
    for i in range(len(idx)):
        for j in range(i + 1, len(idx)):
            stock_i = tickers[idx[i]]
            stock_j = tickers[idx[j]]
            # Check cointegration in-sample
            df_pair = selection_prices[[stock_i, stock_j]].dropna()
            if len(df_pair) < 60:
                continue
            if not is_cointegrated(df_pair[stock_i], df_pair[stock_j]):
                continue
            similarity = corr_matrix.iloc[idx[i], idx[j]]
            selected.append((stock_i, stock_j, similarity))

# Safety cap
max_candidate_pairs = 400
selected = sorted(selected, key=lambda x: abs(x[2]), reverse=True)[:max_candidate_pairs]
selected = pd.DataFrame(selected, columns=["A", "B", "corr"])
print("Candidate pairs after clustering + cointegration:", len(selected))


C:\Users\lenovo ideapad\AppData\Local\Temp\ipykernel_24776\254590658.py:28: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  beta = sm.OLS(y, sm.add_constant(x)).fit().params[1]
C:\Users\lenovo ideapad\AppData\Local\Temp\ipykernel_24776\254590658.py:28: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  beta = sm.OLS(y, sm.add_constant(x)).fit().params[1]
C:\Users\lenovo ideapad\AppData\Local\Temp\ipykernel_24776\254590658.py:28: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value

In [ ]:
def backtest_pair(backtest_prices, A, B, beta,
                  window=60, entry_z=1.6, exit_z=0.5, tc=0.001):
    """
    Backtest a single pair with realistic execution:
    - Lagged signal (t -> t+1)
    - Spread normalization
    - Transaction cost proportional to beta
    """
    
    # Log-price spread
    s = np.log(backtest_prices[A]) - beta * np.log(backtest_prices[B])
    
    # Rolling mean & std
    mean = s.rolling(window).mean()
    std  = s.rolling(window).std()
    
    # Lagged z-score for next-day execution
    z = ((s - mean) / std).shift(1)
    z = z.dropna()
    
    if z.empty:
        return None
    
    pnl = []
    pos_log = []
    last_pos = 0
    
    for i in range(1, len(z)):
        zi = z.iloc[i]
        date = z.index[i]
        
        # Momentum / mean-reversion logic
        if zi > entry_z and last_pos == 0:
            last_pos = 1
            cost = tc * (1 + abs(beta))
        elif zi < -entry_z and last_pos == 0:
            last_pos = -1
            cost = tc * (1 + abs(beta))
        elif abs(zi) < exit_z and last_pos != 0:
            last_pos = 0
            cost = tc * (1 + abs(beta))
        else:
            cost = 0
        
        # Spread change normalized by rolling std
        spread_change = s.iloc[i] - s.iloc[i-1]
        daily_pnl = last_pos * spread_change / std.iloc[i-1] - cost
        
        pnl.append(daily_pnl)
        pos_log.append(last_pos)
    
    pnl = pd.Series(pnl, index=z.index[1:])
    equity = pnl.cumsum()
    
    sharpe = np.sqrt(252) * pnl.mean() / pnl.std() if pnl.std() > 0 else 0
    maxdd = (equity - equity.cummax()).min()
    trades = sum(np.diff([0] + pos_log) != 0) // 2
    
    return {
        "equity": equity,
        "pnl": pnl,
        "sharpe": sharpe,
        "maxdd": maxdd,
        "trades": trades,
        "z": z.reindex(pnl.index)
    }


In [ ]:
# ===============================
# 8️⃣ FINAL REALISTIC OOS BACKTEST
# ===============================

MIN_HOLD_DAYS = 5
TOP_K = 15
MAX_LEVERAGE = 1.25
TARGET_VOL = 0.008

results = []
pair_pnls = []
pair_zscores = []
pair_clusters = {}

# stock → cluster map
stock_cluster_map = dict(zip(tickers, labels))

# ===============================
# Per-pair backtests (signal only)
# ===============================
for _, row in selected.iterrows():
    A, B = row["A"], row["B"]
    similarity = row["corr"]

    cluster_id = stock_cluster_map.get(A)
    if cluster_id is None:
        continue

    # IS beta
    df = selection_prices[[A, B]].dropna()
    X = sm.add_constant(df[B])
    beta = sm.OLS(df[A], X).fit().params.iloc[1]

    if np.isnan(beta) or abs(beta) > 5:
        continue

    res = backtest_pair(
        backtest_prices, A, B, beta,
        window=60, entry_z=1.8, exit_z=0.75, tc=0.001
    )
    if res is None:
        continue

    pair_name = f"{A}-{B}"

    results.append({
        "A": A,
        "B": B,
        "corr": similarity,
        "cluster": cluster_id,
        "sharpe": res["sharpe"],
        "trades": res["trades"]
    })

    pair_pnls.append(res["pnl"].rename(pair_name))
    pair_zscores.append(res["z"].rename(pair_name))
    pair_clusters[pair_name] = cluster_id


# ===============================
# Align data
# ===============================
pair_pnls = pd.concat(pair_pnls, axis=1).fillna(0)
pair_zscores = pd.concat(pair_zscores, axis=1).fillna(0)

# ===============================
# T+1 EXECUTION DELAY
# ===============================
pair_pnls = pair_pnls.shift(1)
pair_zscores = pair_zscores.shift(1)

# ===============================
# DAILY TOP-15 + CLUSTER CAP
# ===============================
final_mask = pd.DataFrame(False, index=pair_pnls.index, columns=pair_pnls.columns)

for date in pair_pnls.index:
    z_today = pair_zscores.loc[date].abs().dropna()
    if z_today.empty:
        continue

    ranked = z_today.sort_values(ascending=False)

    used_clusters = set()
    selected_pairs = []

    for pair in ranked.index:
        cid = pair_clusters[pair]
        if cid in used_clusters:
            continue

        selected_pairs.append(pair)
        used_clusters.add(cid)

        if len(selected_pairs) == TOP_K:
            break

    final_mask.loc[date, selected_pairs] = True

pair_pnls = pair_pnls.where(final_mask, 0)

# ===============================
# MINIMUM HOLDING PERIOD
# ===============================
positions = pair_pnls.copy()
for col in positions.columns:
    active = positions[col] != 0
    hold = 0
    for i in range(len(active)):
        if active.iloc[i]:
            hold += 1
            if hold < MIN_HOLD_DAYS:
                positions.iloc[i, positions.columns.get_loc(col)] = 0
        else:
            hold = 0

pair_pnls = positions

# ===============================
# PORTFOLIO AGGREGATION
# ===============================
portfolio_pnl_raw = pair_pnls.sum(axis=1)

# ===============================
# VOL TARGETING (HARD CAPPED)
# ===============================
realized_vol = portfolio_pnl_raw.rolling(30, min_periods=10).std()
scaling = TARGET_VOL / realized_vol
scaling = (
    scaling.replace([np.inf, -np.inf], np.nan)
           .ffill()
           .fillna(0)
           .clip(0, MAX_LEVERAGE)
)

portfolio_pnl = portfolio_pnl_raw * scaling
portfolio_equity = portfolio_pnl.cumsum()

# ===============================
# METRICS
# ===============================
portfolio_sharpe = (
    np.sqrt(252) * portfolio_pnl.mean() / portfolio_pnl.std()
    if portfolio_pnl.std() > 0 else 0
)
portfolio_maxdd = (portfolio_equity - portfolio_equity.cummax()).min()

# ===============================
# REPORT
# ===============================
print("\n=== FINAL REALISTIC PORTFOLIO (2025 OOS) ===")
print(f"Portfolio Sharpe: {portfolio_sharpe:.3f}")
print(f"Portfolio Max Drawdown: {portfolio_maxdd:.2f}")
print(f"Pairs traded per day (max): {TOP_K}")
print(f"Candidate universe size: {len(results)}")
print(f"Unique clusters used: {len(set(pair_clusters.values()))}")
print(f"Non-zero PnL days: {(portfolio_pnl != 0).sum()} / {len(portfolio_pnl)}")



=== FINAL REALISTIC PORTFOLIO (2025 OOS) ===
Portfolio Sharpe: 1.560
Portfolio Max Drawdown: -0.08
Pairs traded per day (max): 15
Candidate universe size: 120
Unique clusters used: 3
Non-zero PnL days: 59 / 187


In [ ]:

# ===============================
# CREATE PLOTS DIRECTORY
# ===============================
PLOTS_DIR = "plots"
os.makedirs(PLOTS_DIR, exist_ok=True)

# ===============================
# 1️⃣ EQUITY CURVE
# ===============================
plt.figure(figsize=(11, 4))
portfolio_equity.plot()
plt.title("Portfolio Equity Curve (2025 OOS)")
plt.ylabel("Cumulative PnL")
plt.xlabel("Date")
plt.grid(True)
plt.tight_layout()
plt.savefig(f"{PLOTS_DIR}/equity_curve.png", dpi=150)
plt.show()

# ===============================
# 2️⃣ DRAWDOWN CURVE
# ===============================
drawdown = portfolio_equity - portfolio_equity.cummax()

plt.figure(figsize=(11, 3))
drawdown.plot(color="red")
plt.title("Portfolio Drawdown (2025 OOS)")
plt.ylabel("Drawdown")
plt.xlabel("Date")
plt.grid(True)
plt.tight_layout()
plt.savefig(f"{PLOTS_DIR}/drawdown_curve.png", dpi=150)
plt.show()

# ===============================
# 3️⃣ ROLLING SHARPE (60D)
# ===============================
rolling_sharpe = (
    np.sqrt(252)
    * portfolio_pnl.rolling(60).mean()
    / portfolio_pnl.rolling(60).std()
)

plt.figure(figsize=(11, 3))
rolling_sharpe.plot()
plt.axhline(0, color="black", linestyle="--", alpha=0.5)
plt.title("Rolling 60-Day Sharpe Ratio")
plt.ylabel("Sharpe")
plt.xlabel("Date")
plt.grid(True)
plt.tight_layout()
plt.savefig(f"{PLOTS_DIR}/rolling_sharpe_60d.png", dpi=150)
plt.show()

# ===============================
# 4️⃣ EXPANDING SHARPE
# ===============================
expanding_sharpe = (
    np.sqrt(252)
    * portfolio_pnl.expanding().mean()
    / portfolio_pnl.expanding().std()
)

plt.figure(figsize=(11, 3))
expanding_sharpe.plot()
plt.axhline(0, color="black", linestyle="--", alpha=0.5)
plt.title("Expanding Sharpe Ratio")
plt.ylabel("Sharpe")
plt.xlabel("Date")
plt.grid(True)
plt.tight_layout()
plt.savefig(f"{PLOTS_DIR}/expanding_sharpe.png", dpi=150)
plt.show()

# ===============================
# 5️⃣ EXPANDING MAX DRAWDOWN
# ===============================
expanding_maxdd = (
    portfolio_equity - portfolio_equity.cummax()
).expanding().min()

plt.figure(figsize=(11, 3))
expanding_maxdd.plot(color="darkred")
plt.title("Expanding Max Drawdown")
plt.ylabel("Max Drawdown")
plt.xlabel("Date")
plt.grid(True)
plt.tight_layout()
plt.savefig(f"{PLOTS_DIR}/expanding_max_drawdown.png", dpi=150)
plt.show()


NameError: name 'os' is not defined